In [1]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2
from dataset.get_dataset import getDataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torch
import glob

In [2]:
class args_struct:
    def __init__(self):
        self.dataroot = '../../data/DiscRegion/DiscRegion/'
        self.rater_num = 6
        self.img_width = 256
        self.img_height = 256
        self.dataset = 'RIGA'
        self.validate = True
        self.standardize = True
        self.random_seed = 27
        self.loss_func = 'bce'
        self.gt_type_train = -1
args = args_struct()
train_set, valid_set, test_set = getDataset(args, validate=args.validate)

In [3]:
from trainer.train_riga_unet_le import validate_riga_le
from loss_func.get_loss import get_loss_func
args.gt_type_train = -1
args.loss_func = 'bce' 
args.batch_size = 8
args.num_worker = 4
args.use_mix_label = False
loss_func = get_loss_func(args)
from models.unet_le import UNetLE
model = UNetLE()

In [28]:
def do_swa(checkpoint):
	skip = ['relative_position_index', 'num_batches_tracked', 'decoder']

	K = len(checkpoint)
	swa = None
	
	for k in range(K):
		state_dict = torch.load(checkpoint[k], map_location=lambda storage, loc: storage)['model']
		if swa is None:
			swa = state_dict
		else:
			for k, v in state_dict.items():
				# print(k)
				if any(s in k for s in skip): continue
				swa[k] += v
	
	for k, v in swa.items():
		if any(s in k for s in skip): continue
		swa[k] /= K
	
	return swa

In [33]:
path = '/media/kudaibergen/TS512/projects_ts/grader_soup/src/outputs/RIGA_gt_annots_train_majority_vote_UnetLE_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop1/checkpoints/best_loss.pt'
model.load_state_dict(torch.load(path)['model'])

<All keys matched successfully>

In [23]:
checkpoints = ['/media/kudaibergen/TS512/projects_ts/grader_soup/src/outputs/RIGA_gt_annots_train_majority_vote_UnetLE_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop1/checkpoints/best_loss.pt',
               '/media/kudaibergen/TS512/projects_ts/grader_soup/src/outputs/RIGA_gt_annots_train_majority_vote_UnetLE_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt']

In [29]:
swa_weights = do_swa(checkpoints)

In [30]:
model.load_state_dict(swa_weights)

<All keys matched successfully>

In [34]:
validate_riga_le(args, model, valid_set, loss_func, skip_idx=6)

  0%|          | 0/17 [00:00<?, ?it/s]

100%|██████████| 17/17 [00:03<00:00,  5.19it/s]


(0.08715231677244177, 0.9769911438454199, 0.8957165434160306)